In [ ]:
import igl
import math
import scipy as sp
import numpy as np
from meshplot import plot, subplot, interact
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
import os
root_folder = os.getcwd()

In [ ]:
## Load a mesh in OFF format

#v, f = igl.read_triangle_mesh(os.path.join(root_folder, "Coarse_Mesh_Sphere.off"))
###Bending Energy calculation
def fun_EB(v,f):
    k = igl.gaussian_curvature(v, f)
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    minv = sp.sparse.diags(1 / m.diagonal())
    kn = minv.dot(k) 
    area_voronoi=m.diagonal()
    l = igl.cotmatrix(v, f) ###laplacian-operator
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)

    minv = sp.sparse.diags(1 / m.diagonal())

    hn = -minv.dot(l.dot(v))
    h_mean = np.linalg.norm(hn, axis=1)
    c=len(v)
    Kb=1 #bending_modulus
    Eb=[] # bending_energy
    for i in range (c):
        Eb.append((Kb/2)*((h_mean[i])**2)*(area_voronoi[i]))
        Eb_array = np.array(Eb)
    return Eb_array
#print(Eb)

In [ ]:
v, f = igl.read_triangle_mesh(os.path.join(root_folder, "Coarse_Mesh_Sphere.off"))
b=fun_EB(v,f)

np.sum(b)

In [ ]:
##Bending Force Calculation 
def fun_ForceDensity(v,f):

    k = igl.gaussian_curvature(v, f)
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    minv = sp.sparse.diags(1 / m.diagonal())
    kn = minv.dot(k) 
    area_voronoi=m.diagonal()
    l = igl.cotmatrix(v, f) ###laplacian-operator
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)

    minv = sp.sparse.diags(1 / m.diagonal())

    hn = -minv.dot(l.dot(v))
    h_mean = np.linalg.norm(hn, axis=1)
    Lap_H=minv*l*(h_mean/2)
    first_product=h_mean*(((h_mean/2)**2)-k)
    second_product=np.add(Lap_H,first_product)
    n = igl.per_vertex_normals(v, f)
    normal_v=n/np.linalg.norm(n)
    ### Force Density and Nodal Force
    Kb=1
    #Force_density=[] # Force_Density
    Force_Nodal=[]

    #for i in range (len(v)):
    Force_density=2*Kb*(normal_v*second_product[:,None])
        #Force_Nodal.append(Force_density[i]*area_voronoi[i])
        #print(Force_density[i]) 
        #print(Force_Nodal[i])
        
    return Force_density

In [ ]:
#### Random Force Calculation
def fun_FR(v,f):
    gamma=100
    Kb_T=0.01
    del_T=0.01
    sigma=math.sqrt(2*gamma*Kb_T/del_T)
    gaussian_noise=np.random.normal(0, 1, 1)
    adjacent_vertices=igl.adjacency_list(f)
    FRij_nodes=[]
    for i in range(len(v)):
        for j in range(len(adjacent_vertices[i])):
            k=adjacent_vertices[i][j]
            FD_ij=0
            rij=v[i]-v[k]
            rij_norm = rij/np.linalg.norm(rij)
            gaussian_noise=np.random.normal(0, 1, 1)
            FR_ij=FD_ij+(sigma*(np.random.normal(0, 1, 1))*rij_norm)
        FRij_nodes.append(FR_ij)
    return FRij_nodes


In [ ]:
###Calculation of damping" force
# Mesh in (v, f)
def fun_FD(v,f,velocity):
    gamma=100
    Kb_T=0.01
    del_T=0.01
    sigma=math.sqrt(2*gamma*Kb_T/del_T)
    adjacent_vertices=igl.adjacency_list(f)
    FDij_nodes=[]
    for i in range(len(v)):
        for j in range(len(adjacent_vertices[i])):
            FD_ij=0
            k=adjacent_vertices[i][j]
            vij=vel_init[i]-vel_init[k]
            rij=v[i]-v[k]
            #rij_distance=math.sqrt((rij[0][0]**2)+(rij[0][1]**2)+(rij[0][2]**2))
            rij_norm = rij/np.linalg.norm(rij)
            FD_ij=FD_ij+(-((gamma*np.dot(vij, rij)*rij_norm)))
        FDij_nodes.append(FD_ij)
    return FDij_nodes

In [ ]:

m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
minv = np.array(sp.sparse.diags(1 / m.diagonal()))

area_voronoi=np.array(m.diagonal())
vel_init=np.zeros((len(v),3))
#EB=np.array(fun_EB(v,f))
FB=np.array(fun_ForceDensity(v,f)) ##bending_force
FR=np.array(fun_FR(v,f))#random_force
FD=np.array(fun_FD(v,f,vel_init))###dissipative_force

physicalpressure=minv*FB
numericalpressure=minv*(FR+FD)
totalpressure=physicalpressure+numericalpressure



In [ ]:
### Velocity Verlet Integration

dt=0.01
hdt=0.5*0.1
iterations = 1000
timestep = 0.005
T=1
for i in range(iterations):
    for j in range(len(v)):
        v_new=vel_init+0.5*totalpressure*
        

